In [1]:
import numpy as np
from scipy.optimize import fsolve
from scipy.stats import iqr
import pandas as pd
import wbdata as wbd
import holoviews as hv
import hvplot.pandas
import datetime
from statsmodels.tsa.stattools import acf, pacf
from statsmodels.tsa.stattools import adfuller

In [3]:
data_dates = (datetime.datetime(1970,1,1), datetime.datetime(2020,1,1))
indicators={'FP.CPI.TOTL.ZG':'inflation_rate','CM.MKT.TRAD.GD.ZS':'Stocks traded, total value (% of GDP)','CM.MKT.TRNR':'Stocks traded, turnover ratio of domestic shares','NY.GDP.PETR.RT.ZS':"OIL RENT","NY.GDP.MKTP.CD":"GDP CONSTANT 2010 U.S$",'FP.CPI.TOTL':'Consumer Price Index','PA.NUS.PPPC.RF':'Price level ratio','FM.LBL.BMNY.GD.ZS':"BROAD MONEY",'NY.GDP.PCAP.KD':"GDP PER CAPITA CONSTANT 2010 U.S$"}
data = wbd.get_dataframe(indicators, country=['IND'], data_date=data_dates, 
                            convert_date=False, keep_levels=True)[::-1]

In [4]:
data.head(5)

inflation_rate  Stocks traded, total value (% of GDP)  \
country date                                                          
India   1970        5.092262                                    NaN   
        1971        3.079939                                    NaN   
        1972        6.442097                                    NaN   
        1973       16.940816                                    NaN   
        1974       28.598734                                    NaN   

              Stocks traded, turnover ratio of domestic shares  OIL RENT  \
country date                                                               
India   1970                                               NaN  0.046069   
        1971                                               NaN  0.056691   
        1972                                               NaN  0.046521   
        1973                                               NaN  0.056222   
        1974                                               NaN  0.181398   

              GDP CONSTANT 2010 U.S$  Consumer Price Index  Price level ratio  \
country date                                                                    
India   1970            6.242248e+10              4.598095                NaN   
        1971            6.735099e+10              4.739714                NaN   
        1972            7.146319e+10              5.045051                NaN   
        1973            8.551527e+10              5.899724                NaN   
        1974            9.952590e+10              7.586970                NaN   

              BROAD MONEY  GDP PER CAPITA CONSTANT 2010 U.S$  
country date                                                  
India   1970    21.373068                         396.011695  
        1971    23.370552                         393.531274  
        1972    24.466318                         382.450895  
        1973    24.020735                         385.966286  
        1974    22.835456                         381.539586

In [5]:

npa = pd.read_csv("india\Bank Non-Performing Loans to Gross Loans for India.csv")
cpi_all_items = pd.read_csv("india\Consumer Price Index All Items for India.csv")
nomina_GDP = pd.read_csv("india\Current Price Gross Domestic Product in India.csv")
non_banking_credit = pd.read_csv("india\data-Gt630_non banking credit.csv")
cpi_inflation = pd.read_csv("india\Inflation, consumer prices for India.csv")
int_rate_GB = pd.read_csv("india\Interest RatesLong-term government bond yields 10-year Main (including benchmark) for India.csv")
m1_money_supply = pd.read_csv("india\M1 for India.csv")
m3_money_supply = pd.read_csv("india\M3 for India.csv")
property_price_percent_per_annum = pd.read_csv("india\Real Residential Property Prices for India percentperannum.csv")
stock_price_volatility = pd.read_csv("india\Volatility of Stock Price Index for India.csv")
total_share_prices = pd.read_csv("india\Total Share Prices for All Shares for India.csv")
npa.columns=['Date', 'NPA_TO_GROSS_LOAN']
#pd.to_datetime(npa.Date)
#npa.set_index(npa.Date)
npa.head(25)
grp_data = npa
#npa[npa['Date'].str.match('01-01')]

In [7]:
cpi_all_items.columns=['Date', 'CPI']
pd.to_datetime(cpi_all_items.Date)
#cpi_all_items.set_index(cpi_all_items.Date)
cpi_annual = cpi_all_items[cpi_all_items['Date'].str.contains('-01-01')]
cpi_all_items.head()
#cpi_all_items.head()
cpi_annual_1998_ = cpi_annual[(cpi_annual['Date'] >= '1998-01-01')]
cpi_annual_1998_.tail()
grp_data ['CPI'] = list(cpi_annual_1998_.iloc[:-3,1])
cpi_annual_1998_.head()


Date        CPI
456  1998-01-01  31.839964
468  1999-01-01  34.824961
480  2000-01-01  35.737043
492  2001-01-01  36.897875
504  2002-01-01  38.722040

In [9]:
int_rate_GB.columns=['Date', 'INT_RATE']
int_rate_GB.head()

Date  INT_RATE
0  2011-12-01    8.5600
1  2012-01-01    8.2350
2  2012-02-01    8.1925
3  2012-03-01    8.3740
4  2012-04-01    8.5875

In [10]:
nomina_GDP.columns=['Date', 'NOMINA_GDP']
nomina_annual = nomina_GDP[nomina_GDP['Date'].str.contains('-01-01')]
nomina_annual_1998_ = nomina_annual[(nomina_annual['Date'] >= '1998-01-01')]
nomina_annual_1998_.tail()
#nomina_GDP.head()
grp_data ['NOMINA_GDP'] = list(nomina_annual_1998_.iloc[:-2,1])
grp_data.tail()

Date  NPA_TO_GROSS_LOAN         CPI    NOMINA_GDP
15  2013-01-01            4.02773   84.539369  2.605052e+13
16  2014-01-01            4.34585   90.659866  2.940072e+13
17  2015-01-01            5.88352   97.162894  3.214029e+13
18  2016-01-01            9.18552  102.900861  3.588062e+13
19  2017-01-01            9.98000  104.813516  4.004314e+13

In [11]:
non_banking_credit.columns=['Date', 'NON_BANKING_CREDIT']
non_banking_credit.head()

Date  NON_BANKING_CREDIT
0  Mar-2009                9.44
1  Mar-2011               10.81
2  Mar-2013               13.94
3  Mar-2014               14.12
4  Mar-2015               14.52

In [12]:
cpi_inflation.columns=['Date', 'CPI_INFLATION']
cpi_inflation_1998_ = cpi_inflation[(cpi_inflation['Date'] >= '1998-01-01')]
cpi_inflation_1998_.tail()
grp_data ['CPI_INFLATION'] = list(cpi_inflation_1998_.iloc[:-2,1])
grp_data.head()

Date  NPA_TO_GROSS_LOAN        CPI    NOMINA_GDP  CPI_INFLATION
0  1998-01-01               14.4  31.839964  4.060188e+12      13.230839
1  1999-01-01               14.7  34.824961  4.586052e+12       4.669820
2  2000-01-01               12.8  35.737043  5.143873e+12       4.009436
3  2001-01-01               11.4  36.897875  5.429659e+12       3.779293
4  2002-01-01               10.4  38.722040  5.912718e+12       4.297152

In [13]:
m1_money_supply.columns=['Date', 'M1_MONEY']
m1_money_supply_annual = m1_money_supply[m1_money_supply['Date'].str.contains('-01-01')]
m1_money_supply_annual_1998_ = m1_money_supply_annual[(m1_money_supply_annual['Date'] >= '1998-01-01')]
m1_money_supply_annual_1998_.tail()
grp_data ['M1_MONEY'] = list(m1_money_supply_annual_1998_.iloc[:-3,1])
grp_data.head()

Date  NPA_TO_GROSS_LOAN        CPI    NOMINA_GDP  CPI_INFLATION  \
0  1998-01-01               14.4  31.839964  4.060188e+12      13.230839   
1  1999-01-01               14.7  34.824961  4.586052e+12       4.669820   
2  2000-01-01               12.8  35.737043  5.143873e+12       4.009436   
3  2001-01-01               11.4  36.897875  5.429659e+12       3.779293   
4  2002-01-01               10.4  38.722040  5.912718e+12       4.297152   

       M1_MONEY  
0  2.515750e+12  
1  2.877190e+12  
2  3.307250e+12  
3  3.604940e+12  
4  4.005130e+12

In [14]:
m3_money_supply.columns=['Date', 'M3_MONEY']
m3_money_supply_annual = m3_money_supply[m3_money_supply['Date'].str.contains('-01-01')]
m3_money_supply_annual_1998_ = m3_money_supply_annual[(m3_money_supply_annual['Date'] >= '1998-01-01')]
m3_money_supply_annual_1998_.tail()
#m3_money_supply.head()
grp_data ['M3_MONEY'] = list(m3_money_supply_annual_1998_.iloc[:-3,1])
grp_data.head()

Date  NPA_TO_GROSS_LOAN        CPI    NOMINA_GDP  CPI_INFLATION  \
0  1998-01-01               14.4  31.839964  4.060188e+12      13.230839   
1  1999-01-01               14.7  34.824961  4.586052e+12       4.669820   
2  2000-01-01               12.8  35.737043  5.143873e+12       4.009436   
3  2001-01-01               11.4  36.897875  5.429659e+12       3.779293   
4  2002-01-01               10.4  38.722040  5.912718e+12       4.297152   

       M1_MONEY      M3_MONEY  
0  2.515750e+12  7.785870e+12  
1  2.877190e+12  9.402340e+12  
2  3.307250e+12  1.094368e+13  
3  3.604940e+12  1.276376e+13  
4  4.005130e+12  1.458425e+13

In [15]:
property_price_percent_per_annum.columns=['Date', 'PROPERTY_PRICE']
property_price_percent_per_annum.head()

Date  PROPERTY_PRICE
0  2010-01-01            8.24
1  2010-04-01            7.96
2  2010-07-01           11.13
3  2010-10-01            6.66
4  2011-01-01            8.95

In [16]:
stock_price_volatility.columns=['Date', 'VOLATILITY']
stock_price_volatility_1998_ = stock_price_volatility[(stock_price_volatility['Date'] >= '1998-01-01')]
stock_price_volatility_1998_.tail()
#grp_data = grp_data[:-1]
grp_data['VOLATILITY'] = list(stock_price_volatility_1998_.iloc[:,1])
#stock_price_volatility_1998_.tail()
grp_data.head()

Date  NPA_TO_GROSS_LOAN        CPI    NOMINA_GDP  CPI_INFLATION  \
0  1998-01-01               14.4  31.839964  4.060188e+12      13.230839   
1  1999-01-01               14.7  34.824961  4.586052e+12       4.669820   
2  2000-01-01               12.8  35.737043  5.143873e+12       4.009436   
3  2001-01-01               11.4  36.897875  5.429659e+12       3.779293   
4  2002-01-01               10.4  38.722040  5.912718e+12       4.297152   

       M1_MONEY      M3_MONEY  VOLATILITY  
0  2.515750e+12  7.785870e+12     27.1934  
1  2.877190e+12  9.402340e+12     30.4639  
2  3.307250e+12  1.094368e+13     32.5771  
3  3.604940e+12  1.276376e+13     31.8523  
4  4.005130e+12  1.458425e+13     24.3674

In [17]:
total_share_prices.columns=['Date', 'SHARE_PRICE']
total_share_prices_annual = total_share_prices[total_share_prices['Date'].str.contains('-01-01')]
total_share_prices_annual_1998_ = total_share_prices_annual[(total_share_prices_annual['Date'] >= '1998-01-01')]
len(total_share_prices_annual_1998_)
grp_data['SHARE_PRICE'] = list(total_share_prices_annual_1998_.iloc[:-3, 1])
#total_share_prices_annual_1998_.head()
grp_data.head()

Date  NPA_TO_GROSS_LOAN        CPI    NOMINA_GDP  CPI_INFLATION  \
0  1998-01-01               14.4  31.839964  4.060188e+12      13.230839   
1  1999-01-01               14.7  34.824961  4.586052e+12       4.669820   
2  2000-01-01               12.8  35.737043  5.143873e+12       4.009436   
3  2001-01-01               11.4  36.897875  5.429659e+12       3.779293   
4  2002-01-01               10.4  38.722040  5.912718e+12       4.297152   

       M1_MONEY      M3_MONEY  VOLATILITY  SHARE_PRICE  
0  2.515750e+12  7.785870e+12     27.1934    12.694672  
1  2.877190e+12  9.402340e+12     30.4639    12.015583  
2  3.307250e+12  1.094368e+13     32.5771    19.758336  
3  3.604940e+12  1.276376e+13     31.8523    15.206835  
4  4.005130e+12  1.458425e+13     24.3674    12.257635

In [18]:
from sklearn import preprocessing
column_names = ['NPA_TO_GROSS_LOAN', 'CPI', 'NOMINA_GDP', 'CPI_INFLATION','M1_MONEY','M3_MONEY', 'VOLATILITY', 'SHARE_PRICE']
column_names_ = ['NPA_TO_GROSS_LOAN_', 'CPI_', 'NOMINA_GDP_', 'CPI_INFLATION_','M1_MONEY_','M3_MONEY_', 'VOLATILITY_', 'SHARE_PRICE_']
raw = grp_data[column_names]
# Create the Scaler object
scaler = preprocessing.StandardScaler()
# Fit your data on the scaler object
scaled_df = scaler.fit_transform(raw)
scaled_df = pd.DataFrame(scaled_df, columns=column_names_)
scaled_df.head()
grp_data_norm = pd.concat([grp_data, scaled_df], axis=1)
grp_data_norm.head()

Date  NPA_TO_GROSS_LOAN        CPI    NOMINA_GDP  CPI_INFLATION  \
0  1998-01-01               14.4  31.839964  4.060188e+12      13.230839   
1  1999-01-01               14.7  34.824961  4.586052e+12       4.669820   
2  2000-01-01               12.8  35.737043  5.143873e+12       4.009436   
3  2001-01-01               11.4  36.897875  5.429659e+12       3.779293   
4  2002-01-01               10.4  38.722040  5.912718e+12       4.297152   

       M1_MONEY      M3_MONEY  VOLATILITY  SHARE_PRICE  NPA_TO_GROSS_LOAN_  \
0  2.515750e+12  7.785870e+12     27.1934    12.694672            1.814497   
1  2.877190e+12  9.402340e+12     30.4639    12.015583            1.886861   
2  3.307250e+12  1.094368e+13     32.5771    19.758336            1.428555   
3  3.604940e+12  1.276376e+13     31.8523    15.206835            1.090855   
4  4.005130e+12  1.458425e+13     24.3674    12.257635            0.849641   

       CPI_  NOMINA_GDP_  CPI_INFLATION_  M1_MONEY_  M3_MONEY_  VOLATILITY_  \
0 -1.165994    -1.061678        2.113148  -1.210089  -1.086603     0.455913   
1 -1.042173    -1.014570       -0.655461  -1.159180  -1.042647     0.882099   
2 -1.004339    -0.964600       -0.869028  -1.098607  -1.000735     1.157474   
3 -0.956186    -0.938999       -0.943455  -1.056677  -0.951242     1.063024   
4 -0.880518    -0.895726       -0.775981  -1.000311  -0.901739     0.087652   

   SHARE_PRICE_  
0     -1.138734  
1     -1.160774  
2     -0.909474  
3     -1.057198  
4     -1.152918

# Correlation and Covariance between NPA_TO_GROSS_LOAN	CPI	NOMINA_GDP	CPI_INFLATION	M1_MONEY	M3_MONEY	VOLATILITY	and SHARE_PRICE

In [19]:
grp_data_norm[column_names_].corr()

NPA_TO_GROSS_LOAN_      CPI_  NOMINA_GDP_  CPI_INFLATION_  \
NPA_TO_GROSS_LOAN_            1.000000 -0.337603    -0.295851       -0.384903   
CPI_                         -0.337603  1.000000     0.995619        0.057650   
NOMINA_GDP_                  -0.295851  0.995619     1.000000        0.022626   
CPI_INFLATION_               -0.384903  0.057650     0.022626        1.000000   
M1_MONEY_                    -0.442336  0.987502     0.975369        0.146617   
M3_MONEY_                    -0.319406  0.998061     0.998476        0.049319   
VOLATILITY_                   0.074411 -0.622292    -0.626918        0.271296   
SHARE_PRICE_                 -0.459289  0.936812     0.936683        0.130417   

                    M1_MONEY_  M3_MONEY_  VOLATILITY_  SHARE_PRICE_  
NPA_TO_GROSS_LOAN_  -0.442336  -0.319406     0.074411     -0.459289  
CPI_                 0.987502   0.998061    -0.622292      0.936812  
NOMINA_GDP_          0.975369   0.998476    -0.626918      0.936683  
CPI_INFLATION_       0.146617   0.049319     0.271296      0.130417  
M1_MONEY_            1.000000   0.983608    -0.566118      0.950390  
M3_MONEY_            0.983608   1.000000    -0.605532      0.937291  
VOLATILITY_         -0.566118  -0.605532     1.000000     -0.543971  
SHARE_PRICE_         0.950390   0.937291    -0.543971      1.000000

In [20]:
grp_data_norm[column_names_].cov()

NPA_TO_GROSS_LOAN_      CPI_  NOMINA_GDP_  CPI_INFLATION_  \
NPA_TO_GROSS_LOAN_            1.052632 -0.355372    -0.311422       -0.405161   
CPI_                         -0.355372  1.052632     1.048020        0.060684   
NOMINA_GDP_                  -0.311422  1.048020     1.052632        0.023817   
CPI_INFLATION_               -0.405161  0.060684     0.023817        1.052632   
M1_MONEY_                    -0.465617  1.039476     1.026704        0.154333   
M3_MONEY_                    -0.336217  1.050591     1.051027        0.051915   
VOLATILITY_                   0.078327 -0.655045    -0.659914        0.285575   
SHARE_PRICE_                 -0.483462  0.986118     0.985982        0.137281   

                    M1_MONEY_  M3_MONEY_  VOLATILITY_  SHARE_PRICE_  
NPA_TO_GROSS_LOAN_  -0.465617  -0.336217     0.078327     -0.483462  
CPI_                 1.039476   1.050591    -0.655045      0.986118  
NOMINA_GDP_          1.026704   1.051027    -0.659914      0.985982  
CPI_INFLATION_       0.154333   0.051915     0.285575      0.137281  
M1_MONEY_            1.052632   1.035376    -0.595913      1.000410  
M3_MONEY_            1.035376   1.052632    -0.637402      0.986622  
VOLATILITY_         -0.595913  -0.637402     1.052632     -0.572601  
SHARE_PRICE_         1.000410   0.986622    -0.572601      1.052632

In [21]:
#%%opts Bars [width=1000 height=500]
grp_data_norm.hvplot(x='Date', y=column_names_, value_label='Value', rot=90, )

:NdOverlay   [Variable]
   :Curve   [Date]   (Value)

In [22]:
%%opts Bars [width=1000 height=500]
grp_data_norm.hvplot(x='Date', y=column_names_, subplots=True, width=220, height=200, rot=90 )
                

:NdLayout   [Variable]
   :Curve   [Date]   (value)

In [23]:
non_banking_credit.hvplot.bar(x='Date', y='NON_BANKING_CREDIT', rot=90)

:Bars   [Date]   (NON_BANKING_CREDIT)

In [24]:
#%%opts Bars [width=1000 height=500]
grp_data_norm.hvplot(x='Date', y=['NPA_TO_GROSS_LOAN_', 'SHARE_PRICE_'], value_label='Value', rot=90)

:NdOverlay   [Variable]
   :Curve   [Date]   (Value)

## COINTEGRATION OF CONSUMER PRICE INDEX AND NON PERFORMING LOANS

In [43]:
from statsmodels.tsa.stattools import coint
cpi_npl_coint = coint(npa.NPA_TO_GROSS_LOAN, grp_data ['CPI_INFLATION'], trend='c', method='aeg', maxlag=None, autolag='aic', return_results=None)
#len(cpi_annual_1998_.CPI)
cpi_npl_coint

C:\Users\INIKING\anaconda3\lib\site-packages\statsmodels\regression\linear_model.py:1620: RuntimeWarning: divide by zero encountered in double_scalars
  return np.dot(wresid, wresid) / self.df_resid


(-0.0, 0.9859002580259643, array([-4.56572837, -3.67661449, -3.27520568]))

In [25]:
exc_rate = pd.read_csv('indian_rupee_vs_ussd.csv')
inflation_rate = pd.read_csv('india-inflation-rate-cpi_.csv')
pd.to_datetime(inflation_rate.date)
#cpi_all_items.set_index(cpi_all_items.Date)
inflation_rate_annual = inflation_rate[inflation_rate['date'].str.contains('-01-01')]
inflation_rate_annual_1998_ = inflation_rate_annual[(inflation_rate_annual['date'] >= '1998-01-01')]
inflation_rate.tail()

date   Inflation Rate (%)   Annual Change
54  12/31/2014               6.3532           -4.55
55  12/31/2015               5.8724           -0.48
56  12/31/2016               4.9410           -0.93
57  12/31/2017               2.4909           -2.45
58  12/31/2018               4.8607            2.37

In [26]:
exc_rate = exc_rate.dropna()
inflation_rate = inflation_rate.dropna()

inflation_rate.columns = ['date', 'Inflation_rate', 'Annual_Change']

In [27]:
inflation_rate.head()

date  Inflation_rate  Annual_Change
1  12/31/1961          1.6952          -0.08
2  12/31/1962          3.6322           1.94
3  12/31/1963          2.9462          -0.69
4  12/31/1964         13.3553          10.41
5  12/31/1965          9.4748          -3.88

In [28]:
inflation_rate.hvplot(x='date', y='Annual_Change', title="Annual percentage change in inflation rate", rot=90)

:Curve   [date]   (Annual_Change)

# Testing for series STATIONARITY using the ACF and PACF

In [29]:
hv.Bars(acf(inflation_rate.Annual_Change.interpolate().dropna())).redim(y='Autocorrelation', x='Lags')+\
hv.Bars(pacf(inflation_rate.Annual_Change.interpolate().dropna())).redim(y='Patial Autocorrelation', x='Lags').options(color='#FF7E47')

C:\Users\INIKING\anaconda3\lib\site-packages\statsmodels\tsa\stattools.py:572: FutureWarning: fft=True will become the default in a future version of statsmodels. To suppress this warning, explicitly set fft=False.
  FutureWarning


:Layout
   .Bars.I  :Bars   [Lags]   (Autocorrelation)
   .Bars.II :Bars   [Lags]   (Patial Autocorrelation)

# Augmented Dickey-Fuller Test (ADF) Statistical Test
To determine if a time series is stationary or not, we will use the ADF test which is a type of unit root test. Unit roots are a cause for non-stationarity, the ADF test will test if unit root is present.
A time series is stationary if a single shift in time doesn’t change the time series statistical properties, in which case unit root does not exist.
## result
Our p-value is definitely less than 0.5 and is even less than 0.01 so we can say with pretty good confidence that we can reject the null (unit root, non-stationary data) and can assume our data is stationary. Additionally, our ADF is much less than our 1% confidence value of -3.56, so we have another confirmation that we can reject the null.

In [30]:
adf_test = adfuller(inflation_rate.Annual_Change)
print ("ADF = " + str(adf_test[0]))
print ("p-value = " +str(adf_test[1]))
adf_test

ADF = -6.219346986265938
p-value = 5.2697560519786816e-08


(-6.219346986265938,
 5.2697560519786816e-08,
 5,
 52,
 {'1%': -3.562878534649522,
  '5%': -2.918973284023669,
  '10%': -2.597393446745562},
 283.0687104802611)

## running autoregression AR model
THe chosen lag value is 10

In [31]:
train_data = inflation_rate.Annual_Change[1:len(inflation_rate.Annual_Change)-20]
test_data = inflation_rate.Annual_Change[len(inflation_rate.Annual_Change)-20:]

In [32]:
from statsmodels.tsa.ar_model import AR
from sklearn.metrics import r2_score

#train the autoregression model
model = AR(np.array(train_data))
model_fitted = model.fit()

print('The lag value chose is: %s' % model_fitted.k_ar)

The lag value chose is: 9


C:\Users\INIKING\anaconda3\lib\site-packages\statsmodels\tsa\ar_model.py:691: FutureWarning: 
statsmodels.tsa.AR has been deprecated in favor of statsmodels.tsa.AutoReg and
statsmodels.tsa.SARIMAX.

AutoReg adds the ability to specify exogenous variables, include time trends,
and add seasonal dummies. The AutoReg API differs from AR since the model is
treated as immutable, and so the entire specification including the lag
length must be specified when creating the model. This change is too
substantial to incorporate into the existing AR api. The function
ar_select_order performs lag length selection for AutoReg models.

AutoReg only estimates parameters using conditional MLE (OLS). Use SARIMAX to
estimate ARX and related models using full MLE via the Kalman Filter.

To silence this warning and continue using AR until it is removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.ar_model.AR', FutureWarning)

  warnings.warn(AR_DEPRECATION_WARN, FutureWarning)


In [33]:
# make predictions 
predictions = model_fitted.predict(start=len(train_data), end = len(train_data) + len(test_data)-1, dynamic=False)

predictions.view()
# create a comparison dataframe
#compare_df = pd.concat(
#   [inflation_rate.Annual_Change.reset_index().tail(20),
#    predictions], axis=1).rename(
#   columns={'Annual_Change': 'actual', 0:'predicted'})
#compare_df=compare_df[['actual', 'predicted']].dropna()

array([-0.89216989, -2.60943154,  1.65421228, -0.67925772, -2.19009354,
        1.50313075,  2.03831093, -0.70705853, -0.28873975,  1.70684691,
       -0.94877032, -1.78790542,  0.88561236,  0.73797348, -0.40958656,
        0.85747617,  1.05710304, -0.76902947, -0.71300042,  0.48491225])

## VOLATILITY OF TRADED VOLUME OF CURRENCY FROM 2015 to 2018

In [34]:
#exc_rate = exc_rate.set_index('Date')
exc_rate_data_2018 = exc_rate[(exc_rate.Date>='2018-01-01')&(exc_rate.Date<'2019-01-01')]
exc_rate_data_2016 = exc_rate[(exc_rate.Date>='2016-01-01')&(exc_rate.Date<'2017-01-01')]
exc_rate_data_2017 = exc_rate[(exc_rate.Date>='2017-01-01')&(exc_rate.Date<'2018-01-01')]
exc_rate_data_2015 = exc_rate[(exc_rate.Date>='2015-01-01')&(exc_rate.Date<'2016-01-01')]
exc_rate_data_2019 = exc_rate[(exc_rate.Date>='2019-01-01')&(exc_rate.Date<'2020-01-01')]
#exc_rate.head(20)

In [35]:
exc_rate_data_2018.hvplot(x='Date', y='Volume', title="2018")


:Curve   [Date]   (Volume)

In [36]:
data.hvplot(x='date', y='OIL RENT', title='INDIAN OIL RENT', rot=90)

:Curve   [date]   (OIL RENT)